In [ ]:
import json
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def to_csv():
    with open('gpu_price.json', 'r') as file:
        data_dict = json.load(file)
        flattened_data = []   
        for key, model_data in data_dict.items():
            model_data['Model name'] = model_data['Model name']
            flattened_data.append(model_data)
        df = pd.DataFrame(flattened_data)
        df.to_csv('gpu_data.csv', index=False)


In [7]:
df = pd.read_csv('gpu_data.csv')
df.head(5)

,Model name,Launch,Code name,Fab (nm),Transistors (million),Die size (mm),Bus interface,Core clock (MHz),Memory clock (MHz),Core config,...,Branding and Model,Branding and Model Branding and Model.1,MSRP (USD),L2 cache,Clock rate (MHz),Processing power (TFLOPS) Half precision (base),Processing power (TFLOPS) Single precision (base),Processing power (TFLOPS) Double precision (base),Processing power (TFLOPS) XMX Half Precision,Processing power (TFLOPS) Bfloat16
0,NV1,1995-05-22 00:00:00,NV1,SGS 500 nm,1.0,90.0,PCI,75,60,1:1:1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Riva 128,1997-08-25 00:00:00,NV3,SGS 350 nm,4.0,90.0,"AGP 1x, PCI",100,100,1:1:1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Riva 128ZX,1998-02-23 00:00:00,NV3,SGS/TSMC 350 nm,4.0,90.0,"AGP 2x, PCI",100,100,1:1:1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Riva TNT,1998-06-15 00:00:00,NV4,TSMC 350 nm,7.0,90.0,"AGP 2x, PCI",90,110,2:2:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Vanta,1999-03-22 00:00:00,NV6,TSMC 250 nm,NaN,NaN,"AGP 4x, PCI",100,125,2:2:2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
price_cols = [
    c for c in df.columns
    if re.search(r"price|msrp|cost|usd|\$", c, re.IGNORECASE)
]

df[price_cols].isna().mean().sort_values(ascending=False)

MSRP (USD)                              0.994659
Release price (USD) Founders Edition    0.961424
Release price (USD) MSRP                0.951929
Release price (USD)                     0.942433
Release Date & Price                    0.928190
Release Price (USD)                     0.911573
dtype: float64

In [42]:
df_log = df.copy()

for col in price_cols:
    df_log[col] = pd.to_numeric(df_log[col], errors="coerce")
    df_log[col] = df_log[col].apply(lambda x: np.log1p(x) if x > 0 else x)

df[price_cols].describe()
# df_log[price_cols].tail(5)

# plt.figure(figsize=(12, 8))
# for i, col in enumerate(price_cols):
#     plt.subplot(2, 3, i+1)
#     sns.histplot(df_log[col].dropna(), kde=True, color='skyblue')
#     plt.title(f'Log Distribution of {col}')
#     plt.xlabel(f'Log({col})')
#     plt.ylabel('Frequency')
# plt.tight_layout()
# plt.show()

,Release Price (USD),Release price (USD) MSRP,Release price (USD) Founders Edition,Release price (USD),Release Date & Price,MSRP (USD)
count,149,81,65,97,121,9
unique,62,35,19,66,63,8
top,OEM,OEM,-N/a,?,2015,$349
freq,35,7,32,21,19,2
